--- Tweeter parser ---

In [287]:
import tweepy
from tweepy import OAuthHandler
 
consumer_key = 'r8BGSocOtv6vZBnjcwkU5qM4u'
consumer_secret = 'CNAJg23IWXLvUUyJfuFUQOYfzvtKfQVQu6SsY82yJY7GGdhBDC'
access_token = '796083347315367936-grZPgV2cW8DvrozevvihPIJ1Oq9WQVK'
access_secret = 'oJoGFZYDDOXjyG30v0FBChWmGfF9RG6fXvDCVuNrDZJGb'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [288]:
#def process_or_store(tweet):
#    print(json.dumps(tweet))

In [289]:
#import json
#for tweet in tweepy.Cursor(api.user_timeline).items():
#    process_or_store(tweet._json)

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('negative.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

#--- filter for positive tweets 
#twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p'], languages=['fr'])

#--- filter for negative tweets
twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("], languages=['fr'])

In [ ]:
from tweepy import Stream
from tweepy.streaming import StreamListener
 
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('positive.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print('&quot; Error on_data: %s&quot; % str(e)')
            return True
    def on_error(self, status):
        print(status)
        return True
    
twitter_stream = Stream(auth, MyListener())

#--- filter for positive tweets 
twitter_stream.filter(track=[':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p'], languages=['fr'])

#--- filter for negative tweets
#twitter_stream.filter(track=[':(', ':-(', '=(', ')-:', '):', ');', ')=', ":'(", ":'‑("], languages=['fr'])

--- Preprocessing part ---

In [327]:
from nltk.corpus import stopwords
import string

smiles_negativ = [':(', ':-(', '=(', ')-:', '):', ');', ')=']
smiles_positiv = [':)', ':-)', '=)', ':D', ';-)', ';)', '(-;', '(-:', '(:', '(;', '(=', ':d', ':p']
pronoms = ['je', 'tu', 'nous', 'vous', 'il', 'elle', 'ils', 'elles', 'ça', '️']
smiles = smiles_negativ + smiles_positiv
punctuation = list(string.punctuation)
stop = stopwords.words('french') + smiles + ['rt', 'les', '…', 'é', 'ca', 'ée', 'cette', '’', 'a', 'leurs', "'"] + pronoms
stop.remove('pas')

In [328]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

regex_str1 = [
    r"(?:pas\s+[\w_]+)",
    r'(?!pas\s)(?!\s)*tres\s+[\w_]+', # concat "très" + word if there is no "pas" behind
    r'(?!pas\s)(?!\s)*trop\s+[\w_]+', # "trop" + word if there is no "pas" behind] # concat "pas" with the following word
    r'(?!pas\s)(?!\s)*moins\s+[\w_]+'
    r"(?:[a-z][a-z\-_]+[a-z])", # words with - 
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

regex_str_init = [
                r'(?:[\w_]+)',
                r'(?:\S)'
                ]

regex_str2 = [
            r"(?:[\w_]+\s+pas)", # concat "pas" with the preceding word
            ]

regex_str3 = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
]

regex_str4 = [
    r'(?<=pas\s)(?<=\s)*[\w_]+',# looks for the word that follows "pas " (to eliminate it afterwards)
    r'(?<=trop\s)(?<=\s)*[\w_]+',
    r'(?<=tres\s)(?<=\s)*[\w_]+',
    r'(?<=moins\s)(?<=\s)*[\w_]+',
    ] 

tokens_re1 = re.compile(r'('+'|'.join(regex_str1)+')', re.VERBOSE | re.IGNORECASE)
tokens_re2 = re.compile(r'('+'|'.join(regex_str2)+')', re.VERBOSE | re.IGNORECASE)
tokens_re3 = re.compile(r'('+'|'.join(regex_str3)+')', re.VERBOSE | re.IGNORECASE)
tokens_re4 = re.compile(r'('+'|'.join(regex_str4)+')', re.VERBOSE | re.IGNORECASE)

tokens_re_init = re.compile(r'('+'|'.join(regex_str_init)+')', re.VERBOSE | re.IGNORECASE)

In [329]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import FrenchStemmer

def stem_tokens(tokens):
    stemmer = FrenchStemmer()
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def preprocess(s):
    ### HTML tags, @-mentions, # hash-tags, # URLs, # numbers elimination
    s2 = re.sub(tokens_re3, "",s)
    ### devision into words
    tokens_init = tokens_re_init.findall(s2)
    ### stemming
    tokens_init = stem_tokens(tokens_init)
    ### cleaning from stop words
    tokens_init = [term for term in tokens_init if term not in stop]
    toks = tokens_init
    ### string recreation
    s2 = "".join([" "+i for i in tokens_init]).strip()
    ### finding all non matching combinations from tokens_re1 in s2
    tokens1 = tokens_re1.findall(s2)
    ### ...from tokens_re4 in s2
    tokens4 = tokens_re4.findall(s2)
    ### "pas", "trop, "tres", "moins" + word
    tokens2 = tokens_re2.findall(s2)
    tokens = tokens1 + tokens2
    ### cleaning tokens from stop-words and tokens4
    tokens = [term for term in tokens if term not in punctuation and term not in tokens4]
    return tokens

In [330]:
tweet = "Il neige @neige encore ce matin ! :) c'est très beau ! On va avoir une belle hivers on dirait! Nous n'allons pas nous ennuier !!! #hivers #neige"
preprocess(tweet)

['neig',
 'encor',
 'matin',
 'tres beau',
 'va',
 'avoir',
 'bel',
 'hiver',
 'dir',
 'allon',
 'pas ennui',
 'allon pas']

--- Tweets' extracting from the json file ---

In [331]:
import json
import operator
from nltk import bigrams
from collections import Counter

tweets_text = []
mood = []
count_all = Counter()

for line in open('positive.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(1)
    count_all.update(preprocess(text))
  except:
    pass

In [332]:
import json
import operator
from nltk import bigrams
from collections import Counter

for line in open('negative.json'):
  try: 
    new_tweet = json.loads(line)
    text = new_tweet['text']
    tweets_text.append(text)
    mood.append(0)
    count_all.update(preprocess(text))
  except:
    pass

In [333]:
#count_all.most_common(2000)
less_frequent = count_all
for word in list(less_frequent):
    if less_frequent[word] > 5:
        del less_frequent[word]

--- Tf-Idf matrix ---

In [334]:
from sklearn.feature_extraction.text import TfidfVectorizer
stops = less_frequent.keys()
tfidf = TfidfVectorizer(tokenizer=preprocess, stop_words = stops)
A = tfidf.fit_transform(tweets_text)
A = A.toarray()

In [335]:
feature_names = tfidf.get_feature_names()

In [336]:
import numpy as np
A.shape

(24627, 3117)

In [337]:
#U, S, V = np.linalg.svd(A)
#print (U.shape)
#print (S.shape)
#print (V.shape)

In [338]:
#N = 500
#M = dot(dot(U[:,:N],diag(S)[:N,:N]),V[:N,:])

In [339]:
#V2 = V[:500,:]
#M = np.dot(A,V2.transpose())
M = A
M.shape

(24627, 3117)

--- Algorithmic testing part ---

In [340]:
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.cross_validation import train_test_split

X = M
Y = mood

X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [341]:
model = rf()
model.fit(X_train,y_train)
pred = model.predict(X_test)
pred = np.round(pred)

In [342]:
np.sum(np.abs(y_test - pred))/len(y_test)

0.19311352931622544